# Advanced Usage

This tutorial will demonstrate some of the more advanced ``Gryffin`` features. We will introduce sampling strategies and applying multi-dimensional constraints. We will optimize a set of 2 parameters with respect to a 2D *Dejong* surface implemented by the [olympus](https://github.com/aspuru-guzik-group/olympus) package.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%matplotlib

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gryffin import Gryffin
import olympus
from olympus import Surface

Using matplotlib backend: <object object at 0x7fed79ad3240>


Let's again define our objective function and a helper function for evaluation. In this case, our objective function will be given by a 2D *Dejong* surface.

In [2]:
objective = Surface(kind='Dejong', param_dim=2)

def compute_objective(param):
    param['obj'] = objective.run([val for key, val in param.items()])[0][0]
    return param

Let's visualize this objective function:

In [3]:
x_domain = np.linspace(0., 1., 60)
y_domain = np.linspace(0., 1., 60)
X, Y = np.meshgrid(x_domain, y_domain)
Z = np.zeros((x_domain.shape[0], y_domain.shape[0]))

for x_index, x_element in enumerate(x_domain):
    for y_index, y_element in enumerate(y_domain):
        loss_value = objective.run([x_element, y_element])[0][0]
        Z[y_index, x_index] = loss_value

fig, ax = plt.subplots()
contours = plt.contour(X, Y, Z, 3, colors='black')
ax.clabel(contours, inline=True, fontsize=8)
ax.imshow(Z, extent=[0, 1, 0, 1], origin='lower', cmap='RdGy', alpha=0.5)
_  = ax.set_title('Dejong Surface')
_ = ax.set_xlabel('$x_0$')
_ = ax.set_ylabel('$x_1$')


``Gryffin``'s configuration is defined with 2 continuous parameters and a single minimization objective. In the general configuration, we fix the random seed for reproducibility, set verbosity to 0 (to suppress notebook noise). 


In [4]:
config = {
    "general": {
        "random_seed": 42,
        "verbosity": 0,
        "sampling_strategies": 1,
        "boosted":  False,
    },
    "parameters": [
        {"name": "x_0", "type": "continuous", "low": 0.0, "high": 1.0},
        {"name": "x_1", "type": "continuous", "low": 0.0, "high": 1.0},
    ],
    "objectives": [
        {"name": "obj", "goal": "min"},
    ]
}


Here, we overwrite the option ``"sampling_strategies": 2`` in the configuration by passing a specific values for the ``sampling_strategies`` argument to the ``recommend`` method. **TODO - do we define the sampling strategies and then overwrite with the strategy at each timestamp, also can I remove this and add it into the Dejong surface tutorial.**

``Gryffin`` allows us to inject expert knowledge about multi-dimensional contraints on our parameters via a known-contraints function that is passed in a run-time. This is useful to narrow the ``Gryffin`` search space and speed up the optimization. In this case, we know that the sum of our parameters $x_0$ and $x_1$ must be less than $1.2$.

In [5]:
def known_constraints(param):
    return param['x_0'] + param['x_1'] < 1.2

We can visualize this constraint on our objective surface:

In [6]:
fig, ax = plt.subplots()
contours = plt.contour(X, Y, Z, 3, colors='black')
ax.clabel(contours, inline=True, fontsize=8)
ax.imshow(Z, extent=[0, 1, 0, 1], origin='lower', cmap='RdGy', alpha=0.5)

ax.plot(x_domain, 1.2-y_domain, c='k', ls='--', lw=1)
ax.fill_between(x_domain, 1.2-y_domain, 1.2-y_domain+0.8, color='k', alpha=0.4, )
ax.set_ylim(0., 1.)

_ = ax.set_title('Constrained Dejong Surface')
_ = ax.set_xlabel('$x_0$')
_ = ax.set_ylabel('$x_1$')

``Gryffin``'s instance can now be initialized with the configuration above and the constraint function.

In [7]:
gryffin = Gryffin(config_dict=config, known_constraints=known_constraints)

We will now configure two sampling strategies for ``Gryffin``. We will alternate between exploration and exploitation at each iteration of the optimzation. Please refer to the $\lambda$ value presetned in the Phoenics paper for more details. A $\lambda$ or *sampling strategy* value of $-1$ corresponds to exploitation and a value of $1$ corresponds to exploration.

In [8]:
sampling_strategies = [1, -1]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
axes = axes.flatten()
plt.ion()

observations = []
MAX_ITER = 24

for num_iter in range(MAX_ITER):
    print('-'*20, 'Iteration:', num_iter+1, '-'*20)

    # Select alternating sampling strategy (i.e. lambda value presented in the Phoenics paper)
    select_ix = num_iter % len(sampling_strategies)
    sampling_strategy = sampling_strategies[select_ix]

    # Query for new parameters
    params  = gryffin.recommend(
        observations = observations, 
        sampling_strategies=[sampling_strategy]
    )

    param = params[0]
    print('  Proposed Parameters:', param, end=' ')

    # Evaluate the proposed parameters.
    observation = compute_objective(param)
    print('==> Merit:', observation['obj'])
    
    for ax in axes:
        ax.clear()
    if num_iter >=1:
        # plotting ground truth
        x_domain = np.linspace(0., 1., 60)
        y_domain = np.linspace(0., 1., 60)
        X, Y = np.meshgrid(x_domain, y_domain)
        Z    = np.zeros((x_domain.shape[0], y_domain.shape[0]))
        acq = np.zeros((x_domain.shape[0], y_domain.shape[0]))
    
        for x_index, x_element in enumerate(x_domain):
            for y_index, y_element in enumerate(y_domain):
                # evaluate surface
                loss_value = objective.run([x_element, y_element])[0][0]
                Z[y_index, x_index] = loss_value
                # evaluate acquisition function
                acq_value = gryffin.get_acquisition([{'x_0': x_element, 'x_1': y_element }])[sampling_strategy][0]
                acq[y_index, x_index] = acq_value

        contours = axes[0].contour(X, Y, Z, 3, colors='black')
        axes[0].clabel(contours, inline=True, fontsize=8)
        axes[0].imshow(Z, extent=[0, 1, 0, 1], origin='lower', cmap='RdGy', alpha=0.5)

        axes[0].plot(x_domain, 1.2-y_domain, c='k', ls='--', lw=1)
        axes[0].fill_between(x_domain, 1.2-y_domain, 1.2-y_domain+0.8, color='k', alpha=0.4, )

        for obs_index, obs in enumerate(observations):
            if obs_index == 0:
                axes[0].plot(obs['x_0'], obs['x_1'], marker = 'o', color = '#1a1423', markersize = 7, alpha=0.8, label='Previous observations')
            else:
                axes[0].plot(obs['x_0'], obs['x_1'], marker = 'o', color = '#1a1423', markersize = 7, alpha=0.8)

        if len(observations) >= 1:
            # plot the final observation
            axes[0].plot(observations[-1]['x_0'], observations[-1]['x_1'], marker = 'D', color = '#5b2333', markersize = 8, label='Observation')

        axes[0].set_ylim(0., 1.)

        axes[0].set_title('Constrained Dejong surface')
        axes[0].set_ylabel('$x_1$')
        axes[0].set_xlabel('$x_0$')

        axes[0].legend(loc='upper right', fontsize=10)

        # plot acquisition
        contours = axes[1].contour(X, Y, acq, 3, colors='black')
        axes[1].clabel(contours, inline=True, fontsize=8)
        axes[1].imshow(acq, extent=[0, 1, 0, 1], origin='lower', cmap='RdGy', alpha=0.5)

        axes[1].plot(x_domain, 1.2-y_domain, c='k', ls='--', lw=1)
        axes[1].fill_between(x_domain, 1.2-y_domain, 1.2-y_domain+0.8, color='k', alpha=0.4, )

        for obs_index, obs in enumerate(observations):
            if obs_index == 0:
                axes[1].plot(obs['x_0'], obs['x_1'], marker = 'o', color = '#1a1423', markersize = 7, ls='', alpha=0.8, label='Previous observations')
            else:
                axes[1].plot(obs['x_0'], obs['x_1'], marker = 'o', color = '#1a1423', markersize = 7, alpha=0.8)

        if len(observations) >= 1:
            # plot the final observation
            axes[1].plot(observations[-1]['x_0'], observations[-1]['x_1'], marker = 'D', ls='', color = '#5b2333', markersize = 8, label='Observation')

        axes[1].set_ylim(0., 1.)


        axes[1].set_title('Constrained Gryffin acquistion\n' + r'$\lambda=$'+f'{sampling_strategy}')
        axes[1].set_xlabel('$x_0$')



        plt.pause(0.05)
        plt.savefig(f'advanced-usage-vis/iter_{str(num_iter).zfill(2)}')

    # Append this observation to the previous experiments
    observations.append(param)

-------------------- Iteration: 1 --------------------
  Proposed Parameters: {'x_0': 0.03210958, 'x_1': 0.6438679} ==> Merit: 3.362527
-------------------- Iteration: 2 --------------------
  Proposed Parameters: {'x_0': 0.909304141998291, 'x_1': 0.00822313129901886} ==> Merit: 4.240730912022826
-------------------- Iteration: 3 --------------------
  Proposed Parameters: {'x_0': 0.0, 'x_1': 0.5945931673049927} ==> Merit: 3.2086581651173325
-------------------- Iteration: 4 --------------------
  Proposed Parameters: {'x_0': 0.4520935118198395, 'x_1': 0.30063194036483765} ==> Merit: 2.1041226786962097
-------------------- Iteration: 5 --------------------
  Proposed Parameters: {'x_0': 0.569649241516148, 'x_1': 0.27425828788277956} ==> Merit: 2.3370315506847916
-------------------- Iteration: 6 --------------------
  Proposed Parameters: {'x_0': 0.20616485178470612, 'x_1': 0.991956353187561} ==> Merit: 3.9321709465621217
-------------------- Iteration: 7 --------------------
  Propose

We can now visualize this output and observe how the Gryffin aquisition function evolves, alternating between exploration and exploitation at each iteration.

![title](assets/advanced_usage_vis.gif)